In [11]:
# model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

class ConvAutoencoder:
  @staticmethod
  def build(width, height, depth, filters=(32, 64), latent_dim=16):

    # initialize input shape to be channels last along with channels dimension itself
    input_shape = (height, width, depth)
    chan_dim = -1

    # define input to encoder
    inputs = Input(shape=input_shape)
    x = inputs

    # loop over the number of filters
    for f in filters:
      # apply CONV => RELU => BN operations
      x = Conv2D(f, (3, 3), strides=2, padding='same')(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chan_dim)(x)

    # flatten the network and then consturct our latent vector
    volume_size = K.int_shape(x)
    x = Flatten()(x)
    latent = Dense(latent_dim)(x)

    # build the encoder model
    encoder = Model(inputs, latent, name='encoder')

    # start building the decoder model which will accept output of encoder as its input
    latent_input = Input(shape=(latent_dim,))
    x = Dense(np.prod(volume_size[1:]))(latent_input)
    x = Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)

    # loop over number of filters again but this time in reverse order
    for f in filters[::-1]:
      
      # apply a Conv_transpose => RELU => BN operation
      x = Conv2DTranspose(f, (3, 3), strides=2, padding='same')(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chan_dim)(x)

    # apply a single Conv_Transpose layer used to recover the original depth of image
    x = Conv2DTranspose(depth, (3, 3), padding='same')(x)
    outputs = Activation('sigmoid')(x)

    # build decoder model
    decoder = Model(latent_input, outputs, name='decoder')

    # autoencoder is encoder + decoder
    autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')

    return (encoder, decoder, autoencoder)

In [12]:
# training autoencoders

import matplotlib
matplotlib.use('Agg')
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

num_samples = 8          # The number of output samples for visualization
output = 'output.png'    # The path the output visualization image
plot = 'plot.png'        # The path to our matplotlib output plot

epochs = 25
batch_size = 32

# load mnist dataset
print('[INFO] loading MNIST dataset...')
((trainX, _), (testX, _)) = mnist.load_data()

# add channel dimension to every image in dataset and scale pixel intensities to range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

# construct our Conv autoencoder
print('[INFO] building autoencoder...')
(encoder, decoder, autoencoder) = ConvAutoencoder.build(28, 28, 1)
opt = Adam(learning_rate=1e-3)
autoencoder.compile(loss='mse', optimizer=opt)

# train convolution autoencoder
H=autoencoder.fit(trainX, trainX, validation_data=(testX, testX), epochs=epochs, 
                    batch_size=batch_size)

# construct a plot that plots and saves training history
N = np.arange(0, epochs)
plt.style.use('ggplot')
plt.figure()
plt.plot(N, H.history['loss'], label='train_loss')
plt.plot(N, H.history['val_loss'], label='validation_loss')
plt.title('Training loss and accuracy')
plt.xlabel('Epochs #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='lower left')
plt.savefig(plot)

# use convautoencoders to make predictions on test images
print('[INFO] Making predictions...')
decode = autoencoder.predict(testX)
outputs = None

# loop over the number of output samples
for i in range(0, num_samples):
  
  # grab original image and reconstructed image
  original = (testX[i]*255).astype('uint8')
  recon = (decode[i]*255).astype('uint8')

  # stack original and reconstructed image side by side
  output = np.hstack([original, recon])

  # it output array is empty initialize it as current side by side image display
  if outputs is None:
    outputs = output
    # otherwise vertically stack outputs
  else:
    outputs = np.vstack([outputs, output])

# save output image to disk
cv.imwrite('output.png', outputs)

[INFO] loading MNIST dataset...
[INFO] building autoencoder...
Epoch 1/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0190 - val_loss: 0.0112
Epoch 2/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0104 - val_loss: 0.0094
Epoch 3/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0093 - val_loss: 0.0092
Epoch 4/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0087 - val_loss: 0.0087
Epoch 5/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0083 - val_loss: 0.0085
Epoch 6/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0080 - val_loss: 0.0084
Epoch 7/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0078 - val_loss: 0.0077
Epoch 8/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0076 - val_loss: 0.0076
Epoch 9/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0075 - val_loss: 0.0077
Epoch 10/25
1875/1

True